# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_page (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


# ________

In [1]:
# data sample
! hdfs dfs -head /data/clickstream.csv

user_id	session_id	event_type	event_page	timestamp
562	507	page	main	1695584127
562	507	event	main	1695584134
562	507	event	main	1695584144
562	507	event	main	1695584147
562	507	wNaxLlerrorU	main	1695584154
562	507	event	main	1695584154
562	507	event	main	1695584154
562	507	event	main	1695584160
562	507	page	rabota	1695584166
562	507	event	rabota	1695584174
562	507	event	rabota	1695584181
562	507	event	rabota	1695584189
562	507	page	main	1695584194
562	507	event	main	1695584204
562	507	event	main	1695584211
562	507	event	main	1695584211
562	507	event	main	1695584219
562	507	page	bonus	1695584221
562	507	page	online	1695584222
562	507	event	online	1695584230
3539	849	page	main	1695584238
3539	849	event	main	1695584252
3539	849	page	online	1695584261
3539	849	page	bonus	1695584269
3539	849	event	bonus	1695584278
3539	849	page	news	1695584285
3539	849	page	main	1695584291
3539	849	event	main	1695584301
3539	849	page	news	1695584306
3539	849	event	news	1695584307
3539	849	page	vklad	169558

## Solution Prerequisites

 ### Justification

 Following the task description and data structure:
 - User visit to a page always precedes any user action or an error on this particular page
 - User visit to a page can happen without any action on the page but no action on the page can happen without prior user visit to it
 - Therefore, each route is a sequence of pages visited by a user, events do not take part in route composing
 >
 - We are interested in `event_type like %error%` to get a timestamp for breaking user route in case of sessions with errors
 - Even a page on which an error occurred is considered a valid stage of the route if the timestamp of visiting this page is smaller than the timestamp of the error 
 - For valid sessions, i.e. sessions with no error or session parts before the first error, we can obtain route pages by setting `event_type = 'page'`
 - For valid session routes we are no more interested in `event_type` as it is always equal to `'page'` after applied filtering
>
 - Consequent route stages are considered valid even if they are represented by the same page (or pages with coinciding names) 
 - Moreover, the above approach persists even if equally-named route stages happen under the same timestamp
  > Such case was never disclosed in the task description above, so without knowing the nature of web server behavior we cannot make any assumptions in this regard
>
>
- Following the case of coinciding timestamps during same session for same user: 
<br> we have faced an issue when ordering by `user_id`, `session_id`, `timestamp` sometimes does not provide a unique sequence of pages since two pages can be visited under the same timestamp (see the top 2 rows in the data sample below). Additional sorting by `event_type` does not help as this column has the same value for both pages. In order not to limit this solution by an explicit sorting by `event_page` we have introduced a variable `order_by_event_page` which turns ordering by `event_page` on or off depending on the value set.


<pre>
+-------+----------+----------+----------+----------+
|user_id|session_id|event_type|event_page| timestamp|
+-------+----------+----------+----------+----------+
|      0|       901|      page|   tariffs|1698994847|
|      0|       901|      page|    rabota|1698994847|
|      0|       901|     event|   tariffs|1698994869|
|      0|       901|     event|   tariffs|1698994908|
|      0|       901|     event|   tariffs|1698995334|
|      0|       901|     event|   tariffs|1698995461|
|      0|       901|     event|   tariffs|1698995501|
+-------+----------+----------+----------+----------+
</pre>

### General Algorithm
1. Reading the clickstream data
2. Identifying sessions with errors and minimum error timestamps
3. Obtaining user route pages by setting event_types equal to 'page' // filtering out pages visited at the moment or after errors, if any
4. For each pair of (user_id, session_id) getting user routes ordered by timestamp only if `order_by_event_page` set to `False` or both by timestamp and event_page if `order_by_event_page = True` // getting user route counts
5. Selecting 30 most frequent user routes


### Initialization

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

import findspark
findspark.init()

In [3]:
data_path = "hdfs:/data/clickstream.csv"
solution_file_sql = "_sql_result.csv"
solution_file_rdd = "_rdd_result.csv"
solution_file_df = "_df_result.csv"


# SEETINGS for
# file export and ordering
#----------------------------#
order_by_event_page = False   #
single_file_export = False   #
group_file_export = False    #
#----------------------------#

In [ ]:
spark = SparkSession.builder.appName("clickstream").getOrCreate()

# ________

## Spark SQL

In [7]:
sql_string_0 = f"""
    CREATE OR REPLACE TEMPORARY VIEW clickstream_sql
    USING csv
    OPTIONS (
        'path' '{data_path}',
        'header' 'true',
        'sep' '\t'
    )
"""

In [8]:
ordering_string = ', event_page' if order_by_event_page else ''

sql_string_1 = f"""
    WITH 
    error_sessions AS (
        SELECT 
            user_id,
            session_id,
            MIN(timestamp) AS error_timestamp
        FROM 
            clickstream_sql
        WHERE 
            LOWER(event_type) LIKE '%error%'
        GROUP BY 
            user_id, session_id
    ), 
    valid_sessions AS ( 
        SELECT 
            c.user_id,
            c.session_id,
            c.event_page,
            c.timestamp
        FROM 
            clickstream_sql c
        LEFT JOIN 
            error_sessions e
        ON 
            c.user_id = e.user_id AND c.session_id = e.session_id
        WHERE
            c.event_type = 'page' 
            AND (e.error_timestamp IS NULL OR c.timestamp < e.error_timestamp)
    ),
    user_routes AS (
        SELECT DISTINCT
            user_id,
            session_id,
            CONCAT_WS('-', COLLECT_LIST(event_page) OVER (
                PARTITION BY user_id, session_id 
                ORDER BY timestamp{ordering_string} 
                ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
                )
            ) AS route
        FROM 
            valid_sessions
        ORDER BY
            user_id,
            session_id
    )
    SELECT 
        route,
        COUNT(session_id) AS count_sql
    FROM 
        user_routes
    GROUP BY 
        route
    ORDER BY 
        count_sql DESC
    LIMIT 30;
"""

In [ ]:
# 03/03/2024
# aaditional question

sql_string_3 = f"""
    WITH filtered_data AS (
        SELECT 
            user_id,
            session_id,
            event_page,
            timestamp,
            COUNT(CASE WHEN LOWER(event_type) LIKE '%error%' THEN 1 END) OVER (
                PARTITION BY user_id, session_id
                ORDER BY timestamp
                ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
            ) AS error_count
        FROM 
            clickstream_sql
        WHERE 
            event_type = 'page'
    ),
    user_routes AS (
        SELECT DISTINCT
            user_id,
            session_id,
            CONCAT_WS('-', COLLECT_LIST(event_page) OVER (
                PARTITION BY user_id, session_id 
                ORDER BY timestamp{ordering_string} 
                ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
                )
            ) AS route
        FROM 
            filtered_data
        WHERE error_count = 0
    )
    SELECT 
        route,
        COUNT(session_id) AS count_sql
    FROM 
        user_routes
    GROUP BY 
        route
    ORDER BY 
        count_sql DESC
    LIMIT 30;
"""


In [9]:
spark.sql(sql_string_0)
sql_result = spark.sql(sql_string_1)

In [10]:
sql_result.show(30, truncate=False)

+---------------------+---------+
|route                |count_sql|
+---------------------+---------+
|main                 |8090     |
|main-archive         |1096     |
|main-rabota          |1039     |
|main-internet        |880      |
|main-bonus           |865      |
|main-news            |760      |
|main-tariffs         |669      |
|main-online          |584      |
|main-vklad           |514      |
|main-rabota-archive  |167      |
|main-archive-rabota  |167      |
|main-bonus-archive   |139      |
|main-rabota-bonus    |137      |
|main-news-rabota     |134      |
|main-bonus-rabota    |133      |
|main-archive-internet|131      |
|main-rabota-news     |129      |
|main-internet-rabota |128      |
|main-archive-news    |125      |
|main-rabota-internet |123      |
|main-internet-archive|123      |
|main-archive-bonus   |117      |
|main-internet-bonus  |114      |
|main-tariffs-internet|114      |
|main-news-archive    |112      |
|main-news-internet   |108      |
|main-archive-

In [11]:
# -------------------#
# single file export # 
# -------------------#

if single_file_export: sql_result.toPandas().to_csv(solution_file_sql, index=False, sep='\t')

## Spark RDD

In [12]:
clickstream_rdd = (spark
                   .sparkContext.textFile(data_path)
                   .map(lambda line: line.split('\t'))).persist()

error_sessions = (clickstream_rdd
                  .filter(lambda x: 'error' in x[2].lower())
                  .map(lambda x: ((x[0], x[1]), x[4]))
                  .reduceByKey(lambda x, y: min(x, y)))

valid_sessions = (clickstream_rdd
                  .filter(lambda x: x[2].lower() == 'page')
                  .map(lambda x: ((x[0], x[1]), (x[3], x[4])))
                  .leftOuterJoin(error_sessions)
                  .filter(lambda x: x[1][1] is None or (x[1][0][1] < x[1][1])))

if order_by_event_page: 
    user_routes = (valid_sessions
                .map(lambda x: ((x[0][0], x[0][1]), (x[1][0][0], x[1][0][1])))
                .groupByKey()
                .mapValues(lambda x: [page for page, timestamp 
                                        in sorted(x, key=lambda y: (y[1], y[0]))])) 
else:
    user_routes = (valid_sessions
                .map(lambda x: ((x[0][0], x[0][1]), (x[1][0][0], x[1][0][1])))
                .groupByKey()
                .mapValues(lambda x: [page for page, timestamp 
                                        in sorted(x, key=lambda y: (y[1]))])) 


route_counts = (user_routes
                .map(lambda x: (tuple(x[1]), 1))
                .reduceByKey(lambda x, y: x + y))

In [13]:
rdd_result = (spark
              .sparkContext.parallelize(route_counts.takeOrdered(30, key=lambda x: -x[1]))
              .map(lambda x:('-'.join(x[0]), x[1])))

In [14]:
rdd_result.collect()

[('main', 8090),
 ('main-archive', 1096),
 ('main-rabota', 1039),
 ('main-internet', 880),
 ('main-bonus', 865),
 ('main-news', 760),
 ('main-tariffs', 669),
 ('main-online', 584),
 ('main-vklad', 514),
 ('main-rabota-archive', 167),
 ('main-archive-rabota', 167),
 ('main-bonus-archive', 139),
 ('main-rabota-bonus', 137),
 ('main-news-rabota', 134),
 ('main-bonus-rabota', 133),
 ('main-archive-internet', 131),
 ('main-rabota-news', 129),
 ('main-internet-rabota', 128),
 ('main-archive-news', 125),
 ('main-rabota-internet', 123),
 ('main-internet-archive', 123),
 ('main-archive-bonus', 117),
 ('main-tariffs-internet', 114),
 ('main-internet-bonus', 114),
 ('main-news-archive', 112),
 ('main-news-internet', 108),
 ('main-archive-tariffs', 103),
 ('main-tariffs-archive', 102),
 ('main-internet-news', 102),
 ('main-main', 94)]

In [15]:
rdd_result = rdd_result.toDF(["route", "count_rdd"])

In [16]:
rdd_result.show(30, truncate=False)

+---------------------+---------+
|route                |count_rdd|
+---------------------+---------+
|main                 |8090     |
|main-archive         |1096     |
|main-rabota          |1039     |
|main-internet        |880      |
|main-bonus           |865      |
|main-news            |760      |
|main-tariffs         |669      |
|main-online          |584      |
|main-vklad           |514      |
|main-rabota-archive  |167      |
|main-archive-rabota  |167      |
|main-bonus-archive   |139      |
|main-rabota-bonus    |137      |
|main-news-rabota     |134      |
|main-bonus-rabota    |133      |
|main-archive-internet|131      |
|main-rabota-news     |129      |
|main-internet-rabota |128      |
|main-archive-news    |125      |
|main-rabota-internet |123      |
|main-internet-archive|123      |
|main-archive-bonus   |117      |
|main-tariffs-internet|114      |
|main-internet-bonus  |114      |
|main-news-archive    |112      |
|main-news-internet   |108      |
|main-archive-

In [17]:
# -------------------#
# single file export # 
# -------------------#

if single_file_export: rdd_result.toPandas().to_csv(solution_file_rdd, index=False, sep='\t')

## Spark DF

In [18]:
clickstream_df = spark.read.option("delimiter", "\t").csv(data_path, header=True)

In [19]:
error_sessions = (
    clickstream_df
    .filter(F.lower(F.col("event_type")).like("%error%"))
    .groupBy("user_id", "session_id")
    .agg(F.min("timestamp").alias("error_timestamp"))
)

valid_sessions = (
    clickstream_df.alias("c")
    .join(error_sessions.alias("e"), (F.col("c.user_id") == F.col("e.user_id")) & (F.col("c.session_id") == F.col("e.session_id")), "left_outer")
    .filter((F.lower(F.col("c.event_type")) == "page") & ((F.col("e.error_timestamp").isNull()) | (F.col("c.timestamp") < F.col("e.error_timestamp"))))
    .select("c.user_id", "c.session_id", "c.event_page", "c.timestamp")
)

if order_by_event_page: 
    user_routes = (
        valid_sessions
        .withColumn("route", F.collect_list("event_page").over(Window.partitionBy("user_id", "session_id")
                                                            .orderBy("timestamp", "event_page")
                                                            .rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)))
        .select("user_id", "session_id", F.expr("CONCAT_WS('-', route)").alias("route"))
        .distinct()
    )
else:
    user_routes = (
        valid_sessions
        .withColumn("route", F.collect_list("event_page").over(Window.partitionBy("user_id", "session_id")
                                                            .orderBy("timestamp")
                                                            .rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)))
        .select("user_id", "session_id", F.expr("CONCAT_WS('-', route)").alias("route"))
        .distinct()
    )

df_result = (
    user_routes
    .groupBy("route")
    .agg(F.count("session_id").alias("count_df"))
    .orderBy(F.desc("count_df"))
    .limit(30)
)

In [20]:
df_result.show(30, truncate=False)

+---------------------+--------+
|route                |count_df|
+---------------------+--------+
|main                 |8090    |
|main-archive         |1096    |
|main-rabota          |1039    |
|main-internet        |880     |
|main-bonus           |865     |
|main-news            |760     |
|main-tariffs         |669     |
|main-online          |584     |
|main-vklad           |514     |
|main-rabota-archive  |167     |
|main-archive-rabota  |167     |
|main-bonus-archive   |139     |
|main-rabota-bonus    |137     |
|main-news-rabota     |134     |
|main-bonus-rabota    |133     |
|main-archive-internet|131     |
|main-rabota-news     |129     |
|main-internet-rabota |128     |
|main-archive-news    |125     |
|main-rabota-internet |123     |
|main-internet-archive|123     |
|main-archive-bonus   |117     |
|main-internet-bonus  |114     |
|main-tariffs-internet|114     |
|main-news-archive    |112     |
|main-news-internet   |108     |
|main-archive-tariffs |103     |
|main-inte

In [21]:
# -------------------#
# single file export # 
# -------------------#

if single_file_export: df_result.toPandas().to_csv(solution_file_df, index=False, sep='\t')

## Export and Comparison

In [22]:
# ------------------#
# group file export # 
# ------------------#

if group_file_export:
    rdd_result.toPandas().to_csv(solution_file_rdd, index=False, sep='\t')
    rdd_result.toPandas().to_csv(solution_file_rdd, index=False, sep='\t')
    df_result.toPandas().to_csv(solution_file_df, index=False, sep='\t')

In [23]:
pandas_df = sql_result.toPandas().merge(rdd_result.toPandas(), on=['route']).merge(df_result.toPandas(), on=['route'])
pandas_df['equal_results'] = (pandas_df.count_sql.eq(pandas_df.count_rdd) & pandas_df.count_rdd.eq(pandas_df.count_df))

In [24]:
pandas_df

,route,count_sql,count_rdd,count_df,equal_results
0,main,8090,8090,8090,True
1,main-archive,1096,1096,1096,True
2,main-rabota,1039,1039,1039,True
3,main-internet,880,880,880,True
4,main-bonus,865,865,865,True
5,main-news,760,760,760,True
6,main-tariffs,669,669,669,True
7,main-online,584,584,584,True
8,main-vklad,514,514,514,True
9,main-archive-rabota,167,167,167,True


In [25]:
spark.stop()